In [88]:
#makign an Non-deterministic FiniteState Automoton

class diagraph():
    def __init__(self, num_vert, inp=None):
        if inp is not None:
            self.V = inp[0]
            self.E = inp[1]
            self.adj_l = [[] for i in range(self.V)]
            
            for v1,v2 in inp[2:]:
                self.adj_l[v1].append(v2)
#             self.adj_l[v2].append(v1)
        else:
            self.V, self.E = num_vert, 0
            self.adj_l = [[] for i in range(self.V)]
            
    def addEdge(self, v1, v2): 
        self.E += 1
        self.adj_l[v1].append(v2)
        
    def adj(self, v):
        return self.adj_l[v]
    
    def num_vert(self):
        return self.V
    
    def num_edges(self):
        return self.E
    
    def printadj(self):
        for i in range(self.V):
            print(i, self.adj_l[i])
            
class DirectedDFS():
    def __init__(self, diagraph, s):
        self.diagraph = diagraph
        self.s_v = s
        self.marked = [0 for i in range(self.diagraph.V)]

    def reachable(self, s):
        if self.marked[s] == 1:
            return True
        else: return False
            
    def dfs(self, start_vertex=None):
        if start_vertex is None:
            start_vertex = self.s_v 
            
        if type(start_vertex) == int: 
            start_vertex = [start_vertex]
        for v in start_vertex:
            self.marked[v] = 1
            adj_v = self.diagraph.adj(v)
            for vertex in adj_v:
                 if self.marked[vertex] == 0:
                    self.marked[vertex] = 1
                    self.dfs(vertex)

class NFA:
    def __init__(self, re):
        self.re = re
        self.M = len(re)
        self.graph_re = diagraph(self.M+1)
        
    def buildEpsilonTransitionDigraph(self): 
#         graph_re = diagraph(self.M+1)
        stack_ops = []
        
        for i in range(self.M):
            lp = i      
            if (re[i] == '(' or re[i] == '|'): stack_ops.insert(0, i)
       
            elif (re[i] == ')'):
                or_i = stack_ops.pop(0)  
                if (re[or_i] == '|'):
                    lp = stack_ops.pop(0)       #because of 'or', we are now looking for the left paranthesis
                    self.graph_re.addEdge(lp, or_i+1)    
                    self.graph_re.addEdge(or_i, i)
                else: lp = or_i
            
           
            if (i < self.M-1 and re[i+1] == '*'): #closure for star
                self.graph_re.addEdge(lp, i+1)     
                self.graph_re.addEdge(i+1, lp)
                
            if (i < self.M-1 and re[i+1] == '+'): #closure for plus
                self.graph_re.addEdge(lp, i+1)  
                
            if (re[i] == '(' or re[i] == '*' or re[i] == ')' or re[i] == '+'):
                self.graph_re.addEdge(i, i+1) 
#         self.graph_re.printadj()
        return self.graph_re
    
    def recognizes(self, inp):
        pc = []    #possible states  
        dfo = DirectedDFS(self.graph_re, 0)  #allstates reachable by 0 are marked in dfo.marked
        dfo.dfs()
        
        for v in range(self.graph_re.num_vert()):
            if (dfo.marked[v]):
                pc.append(v)
            
        for i in range(len(inp)):    
            match = []     
            for v in pc:        
                if (v == self.M): continue #we are in accept state, so we leave
                if (self.re[v] == inp[i] or re[v] == '.'):  #just looking for matching alphabets      
                    match.append(v+1)
                
            dfo_next = DirectedDFS(self.graph_re, match)
            dfo_next.dfs()
            
            pc = []
                        
            for v in range(self.graph_re.num_vert()):
                if (dfo_next.marked[v]): pc.append(v)
            
            for v in pc:
                if (v == self.M): return True 
        return False
                
re = '(A*B|AC)D'
# re = '(A*C|WB)'
auto = NFA(re)
auto.buildEpsilonTransitionDigraph()
inp = 'ACD'
auto.recognizes(inp)

True

False

In [476]:

#Data compression
#Huffman coding

from collections import Counter

class Node:
    def __init__(self, ch=None, freq=0, left=None, right=None):
        self.ch = ch
        self.freq = freq
        self.left = left
        self.right = right
        
    def isLeaf(self):
        return (self.left == None and self.right == None)
    
    def compareTo(self, x):
        return self.freq - x.freq
    
    

#priority Queue-customized to compare frequencies and sort them in Minheap
class PQ():
    def __init__(self, values=None):
        if values is not None:
            self.pq = self.heap_construction(values, 2)
        else:
            self.pq = []
        
        self.size = len(self.pq)
    
    def insert(self,values):
#             print('values', values)
            self.pq.append(values)
#             print('extending pq valuse', values)
            self.heap_construction(self.pq, 1)
            self.size = len(self.pq)
            
     
    def heap_construction(self, array, ind):
        if array[0] == []:
            pass
        else:
            array.insert(0, [])
    #mid_array here will give us the root of first smallest heap to work with
        mid_array = int((len(array)-1)/2 )     #subtracting 1 because the first element is null

        for i in range(mid_array,0,-1):
            self.sink(i, array, ind)
        self.pq = array
        return array
    
    
    def swim(self, node_ind, array, ind_comp):
        p_node_ind = int(node_ind/2)
        
        while array[node_ind][ind_comp] < array[p_node_ind][ind_comp]:
            temp = array[node_ind]
            array[node_ind] = array[p_node_ind]
            array[p_node_ind] = temp
            node_ind = p_node_ind
            p_node_ind = int(node_ind/2)
            if node_ind < 2:
                break
        return array
    
    def sink(self, node_ind, array, ind_comp):
        size = len(array)
    
        ##check if there are any child nodes
        c_n1, c_n2 = int(2*node_ind), int(2*node_ind + 1)

    
        if c_n1 >= size:
#             print('No child nodes')
            return array
    
        else:
            c_n1v = array[c_n1].freq
            c_n2v = float('inf') if c_n2 >= size else array[c_n2].freq
    
    
        while array[node_ind].freq > c_n1v or array[node_ind].freq > c_n2v:
#             smaller_node_ind = array.index(min(c_n1v, c_n2v), c_n1, c_n2+1)
            min_val = min(c_n1v, c_n2v)
            smaller_node_ind = c_n1 if array[c_n1].freq == min_val else c_n2
            
#             else:
#             smaller_node_ind = next(i for i, lst in enumerate(array[c_n1:c_n2+1]) if array[ind_comp] == min_val)
#         print('Smaller child node and index:', min(c_n1v, c_n2v), smaller_node_ind)
        
            tmp = array[node_ind]
            array[node_ind] = array[smaller_node_ind]
            array[smaller_node_ind] = tmp
        
            node_ind = smaller_node_ind
            c_n1, c_n2 = int(2*node_ind), int(2*node_ind + 1)

            if c_n1 > size or c_n2 >= size or node_ind >= size:
                break
            c_n1v = array[c_n1].freq
            c_n2v = array[c_n2].freq
        
        return array
    
    def printpq(self):
        print('size:', self.size)
        for i in self.pq[1:]:
            print(i.ch)

    
    #Retrieving the minimum and sinking the top value: Exchange top with last and sink the top
    def highest_priority(self):
        array = self.pq
        #here highest priority is the minimum value
        size = len(array)
        smallest = array[1]
        array[1] = array[size - 1]  #last element is now the first one
        array[size-1] = array[1]
        del array[size-1]
        self.sink(1, array, 2)
        self.size -= 1
        return smallest



#implementing Trie for coding. 
class Trie:
    def __init__(self,inp_s):
        self.inp_s = inp_s
#         print('inp_s', self.inp_s)
        
        self.inp_b = []
        for i in inp_s:
            self.inp_b.append(bin(ord(i))[2:].zfill(8))
        self.inp_b = [''.join(str(x) for x in self.inp_b)]
        
        self.en_trie = []
        self.trie_root = None
        self.q =[]

    #build tries by using PQ. uses input in constructor and get frequencies of alphabets in input
    #then uses huffman coding scheme to make a true and saves the root in self.trie_root
    def buildTrie(self): 
        pq = PQ()  
        dic = Counter(self.inp_s)
        for elem in dic.keys():
            pq.insert(Node(elem, dic[elem], None, None))
#         print('pq', pq)
        while (pq.size > 2):
#             pq.printpq()
            x = pq.highest_priority()  
            y = pq.highest_priority()
            parent = Node('\0', x.freq + y.freq, x, y)
            pq.insert(parent)
        self.trie_root = pq.highest_priority()
        return self.trie_root

    #this function writes the trie in bits. uses the preorder to code the bits and nodes.     
    def writeTrie(self, x):
        if (x.isLeaf()):
            self.en_trie.extend('1')
            self.en_trie.extend(x.ch)                                                      
            return
        self.en_trie.extend('0')
        self.writeTrie(x.left) 
        self.writeTrie(x.right)
        me = []
        for i in self.en_trie:
            if i=='0' or i=='1':
                me.append(i)
            else: 
                me.append(bin(ord(i))[2:].zfill(8))          #appending the 8bit value of character
        me = [''.join(str(x) for x in me)]
        return me
    
    #making Symbol table for all the characters in input string
    def buildST(self, x, q, n):
        if (x.isLeaf()):
            q.append([n[:], x.ch])
#             print('inside. found leaf', q)
            n.pop()
            return
    
        n.append(0)
        buildST(x.left, q, n)
        n.append(1)
        buildST(x.right,q, n)
        if len(n) != 0:  n.pop()
            
        return q
   
    #uses ST to encode the message and return the coded bits 
    def encode(self):
        q,n = [],[]
        q = self.buildST(self.trie_root, q, n)
        self.st = {kv[1]: kv[0] for kv in q}
        coded = []
        for i in self.inp_s:
            coded.extend(self.st[i])
            print(i, 'getting coded as', self.st[i])
        coded = [''.join(str(x) for x in coded)]
        return coded
         
    def printall(self, node):
        allnodes = []
        list_nodes = inorder(node, allnodes)
        print( "...".join(list_nodes))
        

        
def inorder(root, allnodes):
    if root:
#         print('root is', root.ch)
        inorder(root.left,allnodes)  
        allnodes.append(root.ch)
        inorder(root.right, allnodes)  
    return allnodes


#inp is binary here coming from the encode message. the following class deals with decoding the message
class expand:
    def __init__(self, inp_s):
        #the following will read the Trie and inp_s has the encoding of the Trie
        self.inp_s = inp_s[0]
        self.str_ptr = 0
        self.root = self.readTrie()
        self.message = []
        
    #reads the trie from the bits sent by writeTrie func. returns the root node.
    def readTrie(self):
        while self.str_ptr>=0 and self.str_ptr < len(self.inp_s):
            if (self.inp_s[self.str_ptr]=='1'):
                c = self.inp_s[self.str_ptr+1: self.str_ptr+9]
                self.str_ptr +=8
                n = int(c, 2)
                v = n.to_bytes((n.bit_length() + 7) // 8, 'big').decode()
                return Node(v, 0, None, None)
                
            self.str_ptr += 1
            x = self.readTrie()
            self.str_ptr += 1
            y = self.readTrie() 
            return Node('\0', 0, x, y)
        
    #decodes the message using the trie read by readTrie
    def decode(self, inp):
        inp = inp[0]
        N = len(inp)
        i=0
        while i < N:
            x = self.root
            while (not x.isLeaf()):
                if (inp[i]== '0'):
                    x = x.left
                else:   
                    x = x.right
                i+=1
            
            self.message.append(x.ch)
        f = [''.join(str(x) for x in self.message)]
        return f
            
    def printall(self, node):
        allnodes = []
        list_nodes = inorder(node, allnodes)
        print( "...".join(list_nodes))
      
data = 'ihis is mama'
print('input', data)
mt = Trie(data)
cod_t = mt.buildTrie()
# mt.printall(cod_t)
c = mt.writeTrie(cod_t)
l = mt.encode()
print(l)
ee = expand(c)
ans=ee.decode(l)
print ('decoded:', ans)
# ee.printall(ee.root)

input ihis is mama
i getting coded as [1, 0]
h getting coded as [0, 1, 0]
i getting coded as [1, 0]
s getting coded as [0, 0]
  getting coded as [1, 1, 1]
i getting coded as [1, 0]
s getting coded as [0, 0]
  getting coded as [1, 1, 1]
m getting coded as [1, 1, 0]
a getting coded as [0, 1, 1]
m getting coded as [1, 1, 0]
a getting coded as [0, 1, 1]
['1001010001111000111110011110011']
decoded: ['ihis is mama']


In [445]:
n = int('01110011', 2)
n.to_bytes((n.bit_length() + 7) // 8, 'big').decode()


# def buildST( x, q, n):
#     if (x.isLeaf()):
#         q.append([n[:], x.ch])
# #         print('inside. found leaf', q)
#         n.pop()
#         return
    
#     n.append(0)
#     buildST(x.left, q, n)
# #     print('q', q, 'n',n)
#     n.append(1)
#     buildST(x.right,q, n)
#     if len(n) != 0:  n.pop()

#     return q
    
# q,n =[], []
# q = buildST(cod_t, q, n)
# print(q)
# q = {kv[1]: kv[0] for kv in q}
# q

's'

In [331]:

#priority Queue
class PQ():
    def __init__(self, values=None):
        if values is not None:
            self.pq = self.heap_construction(values, 2)
        else:
            self.pq = []
        
        self.size = len(self.pq)
    
    def insert(self,values):
            print('values', values)
            self.pq.append(values)
            print('extending pq valuse', values)
            self.heap_construction(self.pq, 1)
            self.size = len(self.pq)
            
     
    def heap_construction(self, array, ind):
        if array[0] == []:
            pass
        else:
            array.insert(0, [])
    #mid_array here will give us the root of first smallest heap to work with
        mid_array = int((len(array)-1)/2 )     #subtracting 1 because the first element is null

        for i in range(mid_array,0,-1):
            self.sink(i, array, ind)
        self.pq = array
#         print('After heap_construction, our pq is ',self.pq)
        return array
    
    
    def swim(self, node_ind, array, ind_comp):
        p_node_ind = int(node_ind/2)
        
        while array[node_ind][ind_comp] < array[p_node_ind][ind_comp]:
            temp = array[node_ind]
            array[node_ind] = array[p_node_ind]
            array[p_node_ind] = temp
            node_ind = p_node_ind
            p_node_ind = int(node_ind/2)
            if node_ind < 2:
                break
        return array
    
    def sink(self, node_ind, array, ind_comp):
        size = len(array)
    
        ##check if there are any child nodes
        c_n1, c_n2 = int(2*node_ind), int(2*node_ind + 1)
        #     print('node indices are ', c_n1, c_n2)
    
        if c_n1 >= size:
#             print('No child nodes')
            return array
    
        else:
            c_n1v = array[c_n1].freq
            c_n2v = float('inf') if c_n2 >= size else array[c_n2].freq
    
    
        while array[node_ind].freq > c_n1v or array[node_ind].freq > c_n2v:
#             smaller_node_ind = array.index(min(c_n1v, c_n2v), c_n1, c_n2+1)
            min_val = min(c_n1v, c_n2v)
            smaller_node_ind = c_n1 if array[c_n1].freq == min_val else c_n2
            
#             else:
#             smaller_node_ind = next(i for i, lst in enumerate(array[c_n1:c_n2+1]) if array[ind_comp] == min_val)
#         print('Smaller child node and index:', min(c_n1v, c_n2v), smaller_node_ind)
        
            tmp = array[node_ind]
            array[node_ind] = array[smaller_node_ind]
            array[smaller_node_ind] = tmp
        
            node_ind = smaller_node_ind
            c_n1, c_n2 = int(2*node_ind), int(2*node_ind + 1)

            if c_n1 > size or c_n2 >= size or node_ind >= size:
                break
            c_n1v = array[c_n1].freq
            c_n2v = array[c_n2].freq
        
        return array
    
    def printpq(self):
        for i in self.pq:
            print(i)

    
    #Retrieving the minimum and sinking the top value: Exchange top with last and sink the top
    def highest_priority(self):
        array = self.pq
#         print('in highestpriorty, array:',array)
        #here highest priority is the minimum value
        size = len(array)
        smallest = array[1]
        array[1] = array[size - 1]  #last element is now the first one
        array[size-1] = array[1]
        del array[size-1]
        self.sink(1, array, 2)
        self.size -= 1
        return smallest

        
def buildTrie(inp): 
    pq = PQ()  
    dic = Counter(inp)
    for elem in dic.keys():
        pq.insert(Node(elem, dic[elem], None, None))
    print('pq', pq)
    while (pq.size > 2):
        x = pq.highest_priority()  
        y = pq.highest_priority()
        print('x', x.ch, x.freq, 'y', y.ch, y.freq)
        parent = Node('\0', dic[x.ch] + dic[y.ch], x, y)
        pq.insert(parent)
    return pq.highest_priority()
n = buildTrie('annbaaaan')

values <__main__.Node object at 0x7f1da8421cc0>
extending pq valuse <__main__.Node object at 0x7f1da8421cc0>
values <__main__.Node object at 0x7f1da8421a58>
extending pq valuse <__main__.Node object at 0x7f1da8421a58>
values <__main__.Node object at 0x7f1da851e438>
extending pq valuse <__main__.Node object at 0x7f1da851e438>
pq <__main__.PQ object at 0x7f1da8421518>
x b 1 y n 3
values <__main__.Node object at 0x7f1da8421a90>
extending pq valuse <__main__.Node object at 0x7f1da8421a90>
x   4 y a 5
values <__main__.Node object at 0x7f1da8421f28>
extending pq valuse <__main__.Node object at 0x7f1da8421f28>


In [264]:
print(bin(ord('l'))[2:].zfill(8))

01101100


In [265]:
from itertools import islice
inp = '001anaum0001hamza01harun' #111100111111111'
for i in inp:
    if i=='1':
        print('inside')
        next(islice(inp, 8), None)
    print(i)


0
0
inside
1
a
n
a
u
m
0
0
0
inside
1
h
a
m
z
a
0
inside
1
h
a
r
u
n


In [142]:

numbers = iter(range(10))
for i in numbers:
    if i == 2:
        next(islice(numbers, 7), None)  # consume 3
    print (i)


0
1
2
4
5
6
7
8
9
